In [ ]:
# Description:  To develop a Program interface that helps in information extraction

In [ ]:
# Title:  YouTube Video Content Analysis and Summarization Automation

In [ ]:
# Main Modules and libraries required: 
# ['BeautifulSoup', 'Contextlib', 'OS', 'Selenium', 'Speech Recognition', 'Time', 'Wave', 'Youtube_dl']

In [ ]:
# Supported languages: ['Indian English', 'American English', 'British English']

In [ ]:
# import libraries
from __future__ import unicode_literals
from collections import defaultdict
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords
import speech_recognition as sr
from string import punctuation
from selenium import webdriver
from pytube import YouTube
from heapq import nlargest
from docx import Document
from gtts import gTTS 
import youtube_dl
import contextlib
import selenium
import splitter
import warnings
import wave
import time
import os
from urllib.parse import urlparse, parse_qs
from docx.enum.text import WD_ALIGN_PARAGRAPH
from nltk.tokenize import sent_tokenize,word_tokenize
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# Ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
# Video details extraction
# ['Video Title','Duration','Published Date','Description','Views Count','Likes Count','Dislikes Count', 'Channel Name', 'Location']
def VideoDetails(link):
    
    wd = webdriver.Chrome(executable_path=r'C:\chrome driver\chromedriver.exe')
    wd.maximize_window()
    time.sleep(1)
    
    wpage = wd.get(link)
    time.sleep(5)
    
    wpage = wd.page_source
    text = bs(wpage)
    time.sleep(2)
    
    wd.close()
    
    #__________________________________________________________________________________________________________________
    
    video_name = text.find("h1", {"class": "style-scope ytd-watch-metadata"}).get_text().lstrip().rstrip()
    
    video_duration = text.find("span", {"class": "ytp-time-duration"}).get_text().lstrip().rstrip()
    
    published_date = 
    
#     published_date = text.find("div", {"id": "date"}).text[1:]
    
    description = text.find("yt-formatted-string", {"class": "content"}).text
    
    views_count = int(''.join([ c for c in text.find("span", attrs={"class": "view-count"}).text if c.isdigit() ]))
    
    likes_count = text.find_all("yt-formatted-string", {"id": "text", "class": "ytd-toggle-button-renderer"})[0].text
    
    dislikes_count = text.find_all("yt-formatted-string", {"id": "text", "class": "ytd-toggle-button-renderer"})[1].text
    
    channel_name = text.find("yt-formatted-string", {"class": "ytd-channel-name"}).find("a").text
    
    location = text.find('yt-formatted-string', {'class':'super-title style-scope ytd-video-primary-info-renderer'}).text
    
    #__________________________________________________________________________________________________________________
    
    return video_name, video_duration, published_date, description, views_count, likes_count, dislikes_count, channel_name, location
    

In [ ]:
# YouTube video ID getter
def get_id(url):
    u_pars = urlparse(url)
    quer_v = parse_qs(u_pars.query).get('v')
    if quer_v:
        return quer_v[0]
    pth = u_pars.path.split('/')
    if pth:
        return pth[-1]
    else:
        print("Invalid YouTube URL")

In [ ]:
# Data cleaning - Normal text to punctuated sentences
def TextToPtext(text):
    try:
        text = text.replace("[Music]", "")
        text = text.replace("[Applause]", "")
        text = text.replace("[ piano pop music, ]")
        text = text.replace("[ kaboom ]")
        text = text.replace("[ lasers. ]")
    except:
        pass  
    
    wd = webdriver.Chrome(executable_path=r'C:\Users\Sravan Kumar\chromedriver.exe')
    wd.maximize_window()
    time.sleep(1)

    wd.get('http://bark.phon.ioc.ee/punctuator')    
    time.sleep(5)  
    
    iText_line = wd.find_element_by_id("input-text")
    time.sleep(1)

    iText_line.send_keys(text)
    time.sleep(2)

    wd.find_element_by_id('punctuate-btn').click()
    time.sleep(5)
    
    while(True):
        pText_Line = wd.find_element_by_id("output-text")
        time.sleep(1)
        pText = pText_Line.get_attribute('textContent')
        if(len(pText)<10):
            time.sleep(3)
        else:
            break

    pText_Line = wd.find_element_by_id("output-text")
    time.sleep(1)

    pText = pText_Line.get_attribute('textContent')
    time.sleep(1)

    wd.close()
    
    return pText 


In [ ]:
# YouTube video's audio download || Transcripts extraction
def VideoToText(url):  
    try:
        video_id = get_id(url)
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        print("Processing...")
        video_text = ''
        for i in range(10**100000):
            try:
                video_text = video_text + transcript_list[i]['text'] + " "
            except:
                break
    except: 
        time.sleep(5)
        ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192'
        }],
        'postprocessor_args': [
            '-ar', '16000'
        ],
        'prefer_ffmpeg': True,
        'keepvideo': False
        }
        
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])                                        

        video_text = RenameAudioFile(url)                                   # Audio renaming
        
    return video_text

In [ ]:
# Rename audio file
def RenameAudioFile(url):
    for file_name in os.listdir(os.getcwd()):
        if file_name.endswith(".wav") or file_name.endswith(".aiff"):
            with contextlib.closing(wave.open(file_name, 'r')) as f:
                frames = f.getnframes()
                rate = f.getframerate()
                duration = frames / float(rate)
                duration = int(duration)
                duration = str(duration)
                new_file_name = 'audio.wav'
                print(file_name)
            os.rename(file_name, new_file_name)            
        else:
            continue

    video_text = AudioToText('audio.wav')                                   # Converting audio to text
    return video_text

In [ ]:
# Audio to text conversion
def AudioToText(audio):   
    filename = audio
    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        try:
            r.adjust_for_ambient_noise(source) 
            audio_data = r.record(source)
            text = r.recognize_google(audio_data) 
        except:
            text = "Process halted due to connection error!"
    try:
        os.remove('audio.wav')
    except:
        pass
    return text

In [ ]:
# Text summarization using NLP
# Text = Video Text + Description
class FrequencySummarizer:
    def __init__(self, min_cut=0.1, max_cut=0.9):
        self.min_cut = min_cut
        self.max_cut = max_cut
        self.stopwords = set(stopwords.words("english")+ list(punctuation))  
        
    def _compute_frequencies(self, word_sent):                               # Compute the frequency of each word
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self.stopwords:
                    freq[word] +=1                                           # Frequencies normalization and filtering
        m = float(max(freq.values()))
        for w in freq.keys():
            freq[w] = freq[w]/m
            if freq[w] >= self.max_cut or freq[w] <= self.min_cut:
                del freq[w]
            return freq
    
    def summarize(self, text, n):
        try:
            sents = sent_tokenize(text)
            assert n <= len(sents)
            word_sent = [word_tokenize(s.lower()) for s in sents]
            self._freq = self._compute_frequencies(word_sent)
            ranking = defaultdict(int)
            for i,sent in enumerate(word_sent):
                for w in sent:
                    if w in self._freq:
                        ranking[i] += self._freq[w]
            sents_idx = self._rank(ranking,n)
            return [sents[j] for j in sents_idx]
        except:
            pass        
    def _rank(self, ranking, n):
            return nlargest(n, ranking, key=ranking.get)

In [ ]:
# Main control
# InpURL = str(input("Enter the YouTube video URL: "))                        # User input: Video URL
InpURL = 'https://youtu.be/B3XY0gmfDHg'
start_time=time.time()
video_name, video_duration, published_date, description, views_count, likes_count, dislikes_count, channel_name, location = VideoDetails(InpURL) 
video_details =  "Video Name         :  " + str(video_name) + "\n" + "Video Duration   :  " + str(video_duration) + "\n" + "Published Date   :  " + str(published_date) + "\n" + "Views Count        :  " + str(views_count) + "\n" + "Likes Count          :  " + str(likes_count) + "\n" + "Dislikes Count     :  " + str(dislikes_count) + "\n" + "Channel Name     :  " + str(channel_name) + "\n" + "Tags                        :  " + str(location) + "\n" + "Description :-  \n" + str(description) + "\n"
video_text = VideoToText(InpURL)
try:
    video_text = TextToPtext(video_text)
except:
    pass
summarized_content = ""
fs = FrequencySummarizer(); 
for ranked_sentence in fs.summarize(video_text,2):
    summarized_content = summarized_content + "\n--> " + ranked_sentence  
end_time=time.time()
print("The present video from the channel:", channel_name, "\b, was processed under",int(end_time-start_time), "seconds")


In [ ]:
# Output Cell
print("".center(122, "-"),"\n", "\b".center(122, "|"), "\n","\b".center(122, "-"), "\n")
print("Video Name       :  ", video_name)
print("Video Duration   :  ", video_duration)
print("Published Date   :  ", published_date)
print("Views Count      :  ", views_count)
print("Likes Count      :  ", likes_count)
print("Dislikes Count   :  ", dislikes_count)
print("Channel Name     :  ", channel_name)
print("Hash tags        :  ", location)
print("Description      :  \n", description)
print("".center(122, "-"),"\n", "\b".center(122, "|"), "\n","\b".center(122, "-"), "\n")
print(video_text)
print("".center(122, "-"),"\n", "\b".center(122, "|"), "\n","\b".center(122, "-"), "\n")
print(summarized_content)
print("".center(122, "-"),"\n", "\b".center(122, "|"), "\n","\b".center(122, "-"), "\n")

In [ ]:
# Saves the collected data into a document/file
document = Document()
document.add_heading("YouTube Video And Content Analysis Summarization Automation", 0)
document.add_heading("YouTube Video Details", level=1)
vd = document.add_paragraph(video_details)
vd.alignment = WD_ALIGN_PARAGRAPH.LEFT
document.add_heading("YouTube Video Text", level=1)
vt = document.add_paragraph(video_text)
vt.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
document.add_heading("YouTube Video Content Summary", level=1)
vs = document.add_paragraph(summarized_content)
vs.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
document.add_page_break()
doc_name = str(input("Enter the file name: ")) + ".docx"
document.save(doc_name)

In [ ]:
# Observations

"""
Links
(1) https://www.youtube.com/watch?v=opixrHfHVDU [3:01] [2:11] [00:50]
(2) https://www.youtube.com/watch?v=Aj9OWdtwDX4 [3:29] [2:59] [00:52]
(3) https://www.youtube.com/watch?v=83R9sFBt3hE [3:16] [2:06] [00:51]
(4) https://www.youtube.com/watch?v=ukzFI9rgwfU [5:00] [4:06] [01:03]
(5) https://www.youtube.com/watch?v=MPWn_or_gvI [2:01] [1:30] [00:52]
(6) https://www.youtube.com/watch?v=Sazgd9KU-Mg [8:33] [6:41] [01:14]
(7) https://www.youtube.com/watch?v=dhYOPzcsbGM [1:49] [1:36] [00:53]
"""
# https://www.youtube.com/watch?v=2AckU9AZc0w
# https://www.youtube.com/watch?v=CQgqGwPzsdc

"""
Average Time Saved: 13x times
"""